In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as pyplot
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

Importing dataset : we have two dataset one contains fake news and other contains true news

In [2]:
df_fake = pd.read_csv('Fake.csv')
df_true = pd.read_csv('True.csv')

In [3]:
df_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
df_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


Inserting a column class as target feature

In [5]:
df_fake['class'] = 0
df_true['class'] = 1

In [6]:
df_true.shape,df_fake.shape

((21417, 5), (23481, 5))

Merging True and Fake dataframes

In [7]:
df_merge = pd.concat([df_fake, df_true], axis = 0)
df_merge.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [8]:
df_merge.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

Remove columns which are not required...

In [9]:
df = df_merge.drop(['title','subject','date'], axis = 1)
df.isnull().sum()

text     0
class    0
dtype: int64

Ramdom shuffle of the dataframe

In [10]:
df = df.sample(frac=1)
df.head()

,text,class
4615,WASHINGTON (Reuters) - The U.S. House voted on...,1
18598,The classless Democrats are at it again. The o...,0
19023,,0
173,"Tomi Lahren, the conservative media pundit, an...",0
18746,BERLIN (Reuters) - Some 59 percent of Germans ...,1


In [11]:
df.reset_index(inplace=True)
df.drop(['index'], axis = 1, inplace = True)

In [12]:
df.columns

Index(['text', 'class'], dtype='object')

In [13]:
df.head()

,text,class
0,WASHINGTON (Reuters) - The U.S. House voted on...,1
1,The classless Democrats are at it again. The o...,0
2,,0
3,"Tomi Lahren, the conservative media pundit, an...",0
4,BERLIN (Reuters) - Some 59 percent of Germans ...,1


creating the function which processes the text....

In [14]:
def wordprocess(text):
    text = text.lower()
    text = re.sub('\[.*?\]','',text)
    text = re.sub("\\W", " ", text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [15]:
df["text"] = df["text"].apply(wordprocess)

In [17]:
#dependent and independent variable
x = df["text"]
y = df["class"]

In [18]:
#split train and test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25)

In [19]:
#initializing a Tfidf Vectorizer
#Tf - term frequency(no of times a word appears in document)
#idf - inverse document frequency(measure of how significant a term in the entire pragraph or corpus.)

from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)